In [1]:
#imports
import pandas as pd
import networkx as nx
import time
import tweepy 

In [2]:
#API Keys
consumer_key = 'HoshUlv1vavF5iYgdNk6TmaOM'
consumer_secret = 'tvFIN4wsy3CnjYWpPX88vLMxb7i5BfLrMyJSm0trPly3R6iBDO'
access_token = '796224491395366914-Tn2B2iBpy6e5xDj5GquZw5P8ap7Lpdi'
access_token_secret = '3csIFTivfgeSP8iEvtOhEzPUH1Pxqzvl6h5gCX0OA4IKQ'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=3, retry_delay=60)

In [4]:
# load data from edges
digwell = pd.read_pickle("digwell.pkl")
digwell.set_index("id", inplace=True)

In [5]:
# make graph with nodelist and node attributes
G = nx.DiGraph() 
G.add_nodes_from(list(digwell.index))
cols = list(digwell.columns)
cols.remove("object")
for col in cols:
    nx.set_node_attributes(G, pd.Series(digwell[col]).to_dict(), col)

In [ ]:
# add edges by iterating through friend lists 

ids = list(digwell.index)

failed = []
j = 0
for i in ids:
    j += 1
    print(str(i) + " " + str(j))
    try:
        following = list(api.friends_ids(user_id = i))
        digwell_follows = list(set(ids) & set(following))
        joined = list(zip([i] * len(digwell_follows), digwell_follows))
        G.add_edges_from(joined)
        nx.write_gpickle(G, "digwelltwitter.gpickle")
    except tweepy.error.TweepError as ex:
        print(ex.reason)
        failed.append(i)

In [ ]:
# second round for failed

for i in failed:
    try:
        following = list(api.friends_ids(user_id = i))
        digwell_follows = list(set(ids) & set (following))
        joined = list(zip([i] * len(digwell_follows), digwell_follows))
        G.add_edges_from(joined)
        nx.write_gpickle(G, "digwelltwitter.gpickle")
    except tweepy.error.TweepError as ex:
        print(ex.reason)
        failed.append(i)

In [ ]:
G.add_edges_from(joined)
nx.write_gpickle(G, "digwelltwitter.gpickle")

In [ ]:
# save edgelist 
edge_df = nx.to_pandas_edgelist(G)
edge_df.to_csv("digwell_edges.csv", index = False) 